## sparkmlhw04
采用ML Pipelines构建一个文档分类器，需要进行文档清洗、提取文档关键字、得到文档向量、提取分类主题的关键字，最后建立一个文档主题目分类器，进行模型训练、测试、分类指标：

In [1]:
df1 = spark.read.csv("/home/hadoop/Desktop/doc_class.dat", sep='|', header=True)

In [2]:
df1.show(1)

+--------+----------+--------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|
+--------+----------+--------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|
+--------+----------+--------+--------------------+--------------------+
only showing top 1 row



In [3]:
#fenci
from pyspark.ml.feature import RegexTokenizer
rtokenizer = RegexTokenizer(inputCol='myapp_word_all', outputCol='words')
rtokenizer.setPattern('[, ]+')
df2 = rtokenizer.transform(df1)
df2.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|
+--------+----------+--------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|
+--------+----------+--------+--------------------+--------------------+--------------------+
only showing top 1 row



In [6]:
#pyspark.ml.feature.StopWordsRemover
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="rwords", stopWords=StopWordsRemover.loadDefaultStopWords('english'))
df3 = remover.transform(df2)
df3.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [7]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(numFeatures=1000, inputCol='rwords', outputCol='features')
df4 = hashingTF.transform(df3)
df4.show(1, False)

+--------+----------+--------+--------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------

In [27]:
#pyspark.ml.feature.Word2Vec
from pyspark.ml.feature import Word2Vec


In [30]:
wv = Word2Vec(vectorSize=128, inputCol='rwords', outputCol='wvfeatures')
wvModel = wv.fit(df3)

In [34]:
wvModel.getVectors().show(1, False)

+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
df5 = wvModel.transform(df3)
df5.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|          wvfeatures|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|[0.02366950090120...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [8]:
#pyspark.ml.feature.CountVectorizer
from pyspark.ml.feature import CountVectorizer


cv = CountVectorizer(vocabSize=1000, inputCol='rwords', outputCol='cvfeatures')
cvModel = cv.fit(df3)

df6 = cvModel.transform(df3)
df6.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|          cvfeatures|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|(510,[0,1,2,3,4,5...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [9]:
from pyspark.ml.clustering import LDA

In [40]:
lda = LDA(k=5)
ldaModel = lda.fit(df4)

In [44]:
ldatransformed = ldaModel.transform(df4)
ldatransformed.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|            features|   topicDistribution|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|(1000,[10,51,68,7...|[0.00407877175527...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [46]:
df5.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|          wvfeatures|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|[0.02366950090120...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [47]:
lda1 = LDA(k=5, featuresCol='wvfeatures')
ldaModel1 = lda1.fit(df5)
ldatransformed1 = ldaModel1.transform(df5)
ldatransformed1.show(1)

Py4JJavaError: An error occurred while calling o721.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 76.0 failed 1 times, most recent failure: Lost task 2.0 in stage 76.0 (TID 170, localhost, executor driver): java.lang.StackOverflowError
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:445)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma

In [11]:
lda2 = LDA(k=5, featuresCol='cvfeatures')
ldaModel2 = lda2.fit(df6)
ldatransformed2 = ldaModel2.transform(df6)
ldatransformed2.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|          cvfeatures|   topicDistribution|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|(510,[0,1,2,3,4,5...|[0.00267203027568...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row

